In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from PIL import Image
import time
import yaml
import numpy as np
from tqdm import tqdm
from torchvision import transforms
from torchvision.transforms import ToPILImage, ToTensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim import Adam
from torchvision.utils import save_image

import models
import utils
from utils import make_coord,set_save_path,ssim
import datasets
from test import eval_psnr,batched_predict

from models import losses
from models.liif import LIIF
from models.discriminator import Discriminator
from models.losses import AdversarialLoss
%matplotlib inline

In [ ]:
modelpath = r'weights\edsr-baseline-liif.pth'
lr_path = r'testimg\div2klrx4\0801x4.png'
hr_path = r'load\div2k\DIV2K_valid_HR\0801.png'
sr_path = r'testimg\ouput.jpg'

In [ ]:
lr = ToTensor()(Image.open(lr_path))
hr = ToTensor()(Image.open(hr_path))

In [ ]:
model = models.make(torch.load(modelpath)['model'], load_sd=True).cuda()

In [ ]:
h, w = lr.shape[1]*2, lr.shape[2]*2

In [ ]:
h,w

In [ ]:
coord = make_coord((h, w)).cuda()

In [ ]:
cell = torch.ones_like(coord)
cell[:, 0] *= 2 / h
cell[:, 1] *= 2 / w

In [ ]:
model

In [ ]:
lr = ((lr - 0.5) / 0.5).cuda().unsqueeze(0)
coord = coord.unsqueeze(0)
cell = cell.unsqueeze(0)

In [ ]:
lr.shape

In [ ]:
coord.shape

In [ ]:
cell.shape

In [ ]:
sr = batched_predict(model, lr, coord, cell, bsize=30000)[0]

In [ ]:
sr = (sr * 0.5 + 0.5).clamp(0, 1).view(h, w, 3).permute(2, 0, 1).cpu()

In [ ]:
ToPILImage()(sr).save(sr_path)
plt.imshow(ToPILImage()(sr))

In [ ]:
ssim(hr.unsqueeze(0),sr.unsqueeze(0))

# 单步

In [ ]:
with torch.no_grad():
    feat = model.gen_feat(lr)

In [ ]:
feat.shape

In [ ]:
for i in range(feat.shape[1]):
    featimg= ToPILImage()(feat[0][i])
    featimg.save(f"feat/featx{i}.jpg")

In [ ]:
featuf = F.unfold(feat, 3, padding=1).view(feat.shape[0], feat.shape[1] * 9, feat.shape[2], feat.shape[3])

In [ ]:
featuf.shape

In [ ]:
for i in range(featuf.shape[1]):
    featimg= ToPILImage()(featuf[0][i])
    featimg.save(f"featunfold/featuf{i}.jpg")

In [ ]:
preds = []
areas = []

In [ ]:
vx_lst = [-1, 1]
vy_lst = [-1, 1]
eps_shift = 1e-6

In [ ]:
rx = 2 / featuf.shape[-2] / 2  #2/LR_H/2
ry = 2 / featuf.shape[-1] / 2  #2/LR_W/2

In [ ]:
feat_coord = make_coord(featuf.shape[-2:], flatten=False).cuda()
#[LR_H,LR_W,2]
feat_coord.shape

In [ ]:
feat_coord = feat_coord.permute(2, 0, 1).unsqueeze(0).expand(featuf.shape[0], 2, *featuf.shape[-2:])
#[N,2,LR_H,LR_W]
feat_coord.shape

In [ ]:
vx = -1
vy = -1

In [ ]:
coord = coord[:, 0: 1000, :]
cell = cell[:, 0: 1000, :]

In [ ]:
coord_ = coord.clone()#[N,SR_H*SR_W,2]
coord_[:, :, 0] += vx * rx + eps_shift
coord_[:, :, 1] += vy * ry + eps_shift
coord_.clamp_(-1 + 1e-6, 1 - 1e-6)

In [ ]:
coord_ = coord_.flip(-1).unsqueeze(1)

In [ ]:
q_feat = F.grid_sample(featuf, coord_, mode='nearest', align_corners=False)
#[N,C*9,1,SR_H*SR_W]

In [ ]:
q_feat = q_feat[:, :, 0, :].permute(0, 2, 1)#[N,SR_H*SR_W,C*9]

In [ ]:
q_coord = F.grid_sample(feat_coord, coord_, mode='nearest', align_corners=False)
#[N,2,1,SR_H*SR_W]

In [ ]:
q_coord = q_coord[:, :, 0, :].permute(0, 2, 1)#[N,SR_H*SR_W,2]

In [ ]:
rel_coord = coord - q_coord #[N,SR_H*SR_W,2]
rel_coord[:, :, 0] *= featuf.shape[-2]
rel_coord[:, :, 1] *= featuf.shape[-1]
inp = torch.cat([q_feat, rel_coord], dim=-1) #[N,SR_H*SR_W,C*9+2]

In [ ]:
rel_cell = cell.clone()
rel_cell[:, :, 0] *= feat.shape[-2]
rel_cell[:, :, 1] *= feat.shape[-1]
inp = torch.cat([inp, rel_cell], dim=-1) #[N,SR_H*SR_W,C*9+2+2]

In [ ]:
model.imnet

In [ ]:
bs, q = coord.shape[:2] #bs=N q=SR_H*SR_W
#[N*SR_H*SR_W,C*9+2+2] --> [N*SR_H*SR_W,3]

In [ ]:
pred = model.imnet(inp.view(bs * q, -1)).view(bs, q, -1) #[N,SR_H*SR_W,3]
preds.append(pred) #[[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W]]

In [ ]:
area = torch.abs(rel_coord[:, :, 0] * rel_coord[:, :, 1])
areas.append(area + 1e-9) #[[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W]]

In [ ]:
tot_area = torch.stack(areas).sum(dim=0) #[N,SR_H*SR_W]

In [ ]:
predimg = (ret * 0.5 + 0.5).clamp(0, 1).view(1000, 1, 3).permute(2, 0, 1).cpu()

In [ ]:
transforms.ToPILImage()(predimg).save("1000x1.jpg")

In [ ]:
if self.local_ensemble:
    t = areas[0]; areas[0] = areas[3]; areas[3] = t #swap(areas[0],areas[3])
    t = areas[1]; areas[1] = areas[2]; areas[2] = t #swap(areas[1],areas[2])
ret = 0
for pred, area in zip(preds, areas):
    ret = ret + pred * (area / tot_area).unsqueeze(-1)

In [ ]:
ret = F.grid_sample(featuf, coord.flip(-1).unsqueeze(1), mode='nearest', align_corners=False)

In [ ]:
ret.shape

In [ ]:
ret = ret[:, :, 0, :].permute(0, 2, 1)

In [ ]:
ret.shape

In [ ]:
ret = ret.permute(0, 2, 1).view(1,576,h,w).squeeze(0)

In [ ]:
ret.shape

In [ ]:
for i in range(ret.shape[0]):
    ToPILImage()(ret[i]).save(f"grid_sample/g_{i}.jpg")

In [ ]:
preds = []
areas = []
for vx in vx_lst:
    for vy in vy_lst:
        coord_ = coord.clone()#[N,SR_H*SR_W,2]
        coord_[:, :, 0] += vx * rx + eps_shift
        coord_[:, :, 1] += vy * ry + eps_shift
        coord_.clamp_(-1 + 1e-6, 1 - 1e-6)
        coord_ = coord_.flip(-1).unsqueeze(1)
        q_feat = F.grid_sample(featuf, coord_, mode='nearest', align_corners=False)
        #[N,C*9,1,SR_H*SR_W]
        q_feat = q_feat[:, :, 0, :].permute(0, 2, 1)#[N,SR_H*SR_W,C*9]
        q_coord = F.grid_sample(feat_coord, coord_, mode='nearest', align_corners=False)
        #[N,2,1,SR_H*SR_W]
        q_coord = q_coord[:, :, 0, :].permute(0, 2, 1)#[N,SR_H*SR_W,2]

        rel_coord = coord - q_coord #[N,SR_H*SR_W,2]
        rel_coord[:, :, 0] *= feat.shape[-2]
        rel_coord[:, :, 1] *= feat.shape[-1]
        inp = torch.cat([q_feat, rel_coord], dim=-1) #[N,SR_H*SR_W,C*9+2]

        rel_cell = cell.clone()
        rel_cell[:, :, 0] *= feat.shape[-2]
        rel_cell[:, :, 1] *= feat.shape[-1]
        inp = torch.cat([inp, rel_cell], dim=-1) #[N,SR_H*SR_W,C*9+2+2]

        bs, q = coord.shape[:2] #bs=N q=SR_H*SR_W
        #[N*SR_H*SR_W,C*9+2+2] --> [N*SR_H*SR_W,3]
        pred = model.imnet(inp.view(bs * q, -1)).view(bs, q, -1) #[N,SR_H*SR_W,3]
        preds.append(pred) #[[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W]]

        area = torch.abs(rel_coord[:, :, 0] * rel_coord[:, :, 1])
        areas.append(area + 1e-9) #[[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W],[N,SR_H*SR_W]]

tot_area = torch.stack(areas).sum(dim=0) #[N,SR_H*SR_W]

t = areas[0]; areas[0] = areas[3]; areas[3] = t #swap(areas[0],areas[3])
t = areas[1]; areas[1] = areas[2]; areas[2] = t #swap(areas[1],areas[2])
ret = 0
for pred, area in zip(preds, areas):
    ret = ret + pred * (area / tot_area).unsqueeze(-1)